# import functions

In [87]:
import mysql.connector
from sqlalchemy import create_engine
import pandas as pd
from mysql.connector import Error

# Specifying credentials, create snd connect to database

In [93]:
host = '34.87.87.119'
user = 'bt4301_root'
passwd = 'bt4301ftw'
database='bt4301_gp_datawarehouse'
port = '3306'

In [89]:
def create_database_if_not_exists():
    try:
        # Connect to the MySQL server
        connection = mysql.connector.connect(
            host=host,
            user=user,  # Replace with your MySQL username
            password=passwd  # Replace with your MySQL password
        )

        # Create a cursor object
        cursor = connection.cursor()
        
        # Check if the database exists
        cursor.execute("SHOW DATABASES")
        databases = [db[0] for db in cursor.fetchall()]
        if database in databases:
            print(f"Database '{database}' already exists. No need to create it.")
        else:
            # Execute a query to create a database
            cursor.execute(f"CREATE DATABASE {database}")
            print(f"Database '{database}' created successfully")

    except Error as e:
        print(f"Error: {e}")
    
    finally:
        # Close the connection and cursor
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# Call the function
create_database_if_not_exists()

Database 'bt4301_gp_datawarehouse' already exists. No need to create it.
MySQL connection is closed


# housekeeping

In [91]:
db_datawarehouse = mysql.connector.connect(
	host=host,
	user=user,
	passwd=passwd,
	database=database
)

cursor = db_datawarehouse.cursor()
cursor.execute('DROP TABLE IF EXISTS reviews_fact;')
cursor.execute('DROP TABLE IF EXISTS airline_dimension;')
cursor.execute('DROP TABLE IF EXISTS flight_dimension;')


db_datawarehouse.commit()
db_datawarehouse.close()

In [94]:
engine = create_engine(f'mysql://{user}:{passwd}@{host}:{port}/{database}?charset=utf8mb4', echo=False,future=True)
db_sent = engine.connect()

# Prepare data to insert into database

In [95]:
# to change to get csv from datalake
raw_data = pd.read_csv('all_reviews.csv')

In [96]:
reviews_fact = [
    "Review ID",
    "Date Published",
    "Overall Rating",
    "Passenger Country",
    "Trip Verified",
    "Review",
    "Airline",
    "Type Of Traveller",
    "Seat Type",
    "Seat Comfort",
    "Cabin Staff Service",
    "Food & Beverages",
    "Ground Service",
    "Wifi & Connectivity",
    "Value For Money",
    "Recommended",
    "Inflight Entertainment"]

# "FlightID" to be created, not in raw_data

## Reviews Fact

In [97]:
reviews_fact = raw_data[reviews_fact]

In [98]:
from datetime import datetime
reviews_fact['Date Published'] = pd.to_datetime(reviews_fact['Date Published'])

/var/folders/tr/5jr_r6nj3ng65wvq5fs4q41w0000gn/T/ipykernel_81896/1584447569.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_fact['Date Published'] = pd.to_datetime(reviews_fact['Date Published'])


In [99]:
reviews_fact.to_sql(name='reviews_fact', con=db_sent, if_exists='replace')

db_sent.commit()

## Airline Dimension

In [100]:
airline_dimension = raw_data[['Airline','Overall Rating']].groupby(by=['Airline']).mean()

In [101]:
from sqlalchemy.types import VARCHAR
sql_types = {
    'Airline': VARCHAR(255)  # or another length that suits your data
}

airline_dimension.to_sql(name='airline_dimension', con=db_sent, if_exists='replace',dtype=sql_types)

db_sent.commit()

## Flight Dimension

In [102]:
raw_data.columns

Index(['Airline', 'Review ID', 'Date Published', 'Overall Rating',
       'Passenger Country', 'Trip Verified', 'Review Title', 'Review',
       'Aircraft', 'Type Of Traveller', 'Seat Type', 'Origin', 'Destination',
       'Layover', 'Date Flown', 'Seat Comfort', 'Cabin Staff Service',
       'Food & Beverages', 'Ground Service', 'Wifi & Connectivity',
       'Value For Money', 'Recommended', 'Inflight Entertainment'],
      dtype='object')

In [103]:
flight_dim_list = ['Origin','Destination','Layover','Aircraft','Date Flown']

In [104]:
flight_dimension = raw_data[flight_dim_list].drop_duplicates()
flight_dimension['Date Flown'] = pd.to_datetime(flight_dimension['Date Flown'])

In [105]:
flight_dimension.to_sql(name='flight_dimension', con=db_sent, if_exists='replace')

db_sent.commit()

In [106]:
db_sent.close()

# Set Keys

In [107]:
db_sent = mysql.connector.connect(
	host=host,
	user=user,
	passwd=passwd,
	database=database
)
cursor = db_sent.cursor()
cursor.execute('ALTER TABLE airline_dimension MODIFY `Airline` VARCHAR(255);')
cursor.execute('ALTER TABLE reviews_fact MODIFY `Airline` VARCHAR(255);')
cursor.execute('ALTER TABLE reviews_fact ADD PRIMARY KEY (`Review ID`);')
cursor.execute('ALTER TABLE airline_dimension ADD PRIMARY KEY (`Airline`(255))')
cursor.execute('ALTER TABLE flight_dimension ADD PRIMARY KEY (`index`)')
cursor.execute('ALTER TABLE reviews_fact ADD FOREIGN KEY (`Airline`) REFERENCES airline_dimension(`Airline`);')

db_sent.commit()
db_sent.close()